
# Requried libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import netCDF4 as nc
from netCDF4 import Dataset
import math
from scipy import ndimage
from skimage import measure
from statistics import mode, mean
import shutil
import os

# TIER preprocessing driver

 This is the main preprocessing script to generate the grid and TIER specific 
 input station list files for input to TIER

 Three input data requirements not covered in this preprocessing suite:

1. The mask in the raw DEM MUST be determined by the user.  This mask determines land/ocean/not valid and is NOT developed here.  This mask is used for computations (valid land pixels only) and also the distance to coast calculation.  The distance to coast estimates the distance to/from bodies of water that may impart moisture to influence precipitation nearby.  Thus the user can determine which bodies of water receive the 'ocean' flag and are deemed important.  A smaller lake that is flagged as not valid may/may not influence precipitation, but the Great Lakes do influence local precipitation.

Of course the user is encouraged to change/experiment with which pixels 
values are computed over and/or how distance to coast is computed.

2. The user MUST have the input station data in the proper netcdf file format. That can easily be copied from the example data.

3. If the user desires a spatailly variable default lapse rate (right now for temperature only) that needs to be provided as well in a netcdf file. The user can again follow the provided example data set.

Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
Email : anewman@ucar.edu & mozhgana@ucar.edu
 
Copyright (C) 2019 University Corporation for Atmospheric Research

This file is part of TIER.

TIER is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

TIER is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with TIER.  If not, see <https://www.gnu.org/licenses/>.



# User determines the controlName

In [2]:
controlName = input('Enter the name of your control file: ')

Enter the name of your control file: tierPreprocessControl.txt


# readPreprocessControl:
* **Reads a text control file for TIER preprocessing**

In [3]:
def readPreprocessControl(controlName):
    """"
    readPreprocessControl reads a text control file for TIER preprocessing
    TIER - Topographically InformEd Regression

     Arguments:

     Input: controlName, string, the name of the grid file

     Output:controlVars, structure, stucture holding all preprocessing control variables

     Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
     Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """

    #open control file
    fid = open(controlName,'r')
    #read data
    data = np.loadtxt(fid, delimiter=',',skiprows=1,dtype=str, usecols=[0,1,2])
    #close control file
    fid.close()

    class structtype():
        pass
    controlVars = structtype()

    #run through all lines in control file
    for i in range(0,len(data)):
        #test string name and place in appropriate named variable
        if [row[0] for row in data][i]== 'rawGridName':
            controlVars.gridName = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='outputGridName':
            controlVars.outputName = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='preprocessParameterFile':
            controlVars.parameterFile = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='stationPrecipPath':
            controlVars.stationPrecipPath = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='stationTempPath':
            controlVars.stationTempPath= [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='stationPrecipListName':
            controlVars.stationPrecipListName = [row[1] for row in data][i].strip()

        elif [row[0] for row in data][i]=='stationTempListName':
            controlVars.stationTempListName = [row[1] for row in data][i].strip()

        else:
            #throw error if unknown string
            print('Unknown control file option: ' +[row[0] for row in data][i])

    return controlVars

# initPreprocessParameters:
* **Initalizes TIER parameters to defaults**

In [173]:
def initPreprocessParameters():
    """
    initPreprocessParameters initalizes TIER parameters to defaults
     TIER - Topographically InformEd Regression

     Arguments:

        Output:parameters, structure, structure holding all TIER preprocessing parameters

     Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
     Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    class structtype():
        pass
    parameters = structtype()
    #initialize all parameters to initial default value
    parameters.demFilterName = 'Daly'          #string
    parameters.demFilterPasses = 80            #number
    parameters.minGradient = 0.003             #km/km
    parameters.smallFacet = 500                #km^2
    parameters.smallFlat = 1000                #km^2
    parameters.narrowFlatRatio = 3.1           #ratio
    parameters.coastSearchLength = 200         #km
    parameters.layerSearchLength = 10          #grid cells
    parameters.inversionHeight = 250           #m

    return parameters          


In [175]:
# read control file
controlVars = readPreprocessControl(controlName)
# initialize parameter structure to default values
parameters = initPreprocessParameters()

parameters.demFilterPasses

80

# readPreprocessParameters:
* **Reads a text parameter file for TIER preprocessing**
* **Overrides the default values if parameters are present in parameter file**

In [176]:
def readPreprocessParameters(parameterFile,parameters):
    """
    readPreprocessParameters reads a text parameter file for TIER preprocessing
      and overrides the default values if parameters are present
      in parameter file

     TIER - Topographically InformEd Regression

     Arguments:

     Input:parameterFile, string, the name of the TIER preprocessing parameter file
              parameters, structure, structure holding all TIER preprocessing parameters

     Output:parameters, structure, structure holding all TIER preprocessing parameters

     Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
     Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #open parameter file
    fid = open(parameterFile,'r')
    #read data
    data = np.loadtxt(fid, delimiter=',',skiprows=1,dtype=str, usecols=[0,1,2])
    #close control file
    fid.close()   
    
    #run through all lines in parameter file
    for i in range(0,len(data)):
        #test string name and place in appropriate named variable
        if [row[0] for row in data][i]== 'demFilterName':
            #filter type (Daly = original Daly et al. 1994 filter), only option currently implemented
            parameters.demFilterName = [row[1] for row in data][i].strip()
        elif [row[0] for row in data][i]== 'demFilterPasses':
            #number of passes to filter raw DEM
            parameters.demFilterPasses = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='minGradient':
            #minimum gradient for a pixel to be considered sloped, otherwise it is considered flat
            parameters.minGradient = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='smallFacet':
            #area of smallest sloped facet allowed (km^2)
            parameters.smallFacet = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='smallFlat':
            #area of smallest flat facet allowed (km^2)
            parameters.smallFlat = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='narrowFlatRatio':
            #ratio of major/minor axes to merge flat regions (i.e. ridges)
            parameters.narrowFlatRatio = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='layerSearchLength':
            #search length (grid cells) to determine local minima in elevation
            parameters.layerSearchLength = float([row[1] for row in data][i].strip())
        elif [row[0] for row in data][i]=='inversionHeight':
            #depth of layer 1 (temperature inversion layer) in m
            parameters.inversionHeight = float([row[1] for row in data][i].strip())
        else:
            #throw error if unknown string
            print('Unknown parameter name : '+[row[0] for row in data][i])
                
    return parameters


In [177]:
#read preprocessing parameters file
parameters = readPreprocessParameters(controlVars.parameterFile,parameters)
parameters.demFilterPasses

8.0

# readRawGrid:
* **Reads a netcdf grid file for the TIER preprocessing code**


In [178]:
def readRawGrid(gridName):
    """
    readRawGrid reads a netcdf grid file for the TIER preprocessing code
    TIER - Topographically InformEd Regression

     Arguments:

     Input:gridName, string, the name of the grid file

     Output:grid, structure, structure holding DEM and related variables

     Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
     Email : anewman@ucar.edu & mozhgana@ucar.edu


     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    
    class structtype():
        pass
    grid = structtype()
    
    #loading dataset by passing a NetCDF file path
    ds = Dataset(gridName)
    
    #read latitude and longitude
    grid.lat = ds['latitude'][:]
    grid.lon = ds['longitude'][:]
    #grid spacing
    gridUnits = ds['dx'].units
    
    if gridUnits.lower()=='degrees':
        grid.dx = ds['dx'][:]
        #convert to km (roughly)
        
        #grid lower left lat,lon
        startLon = ds['startx'][:]
        startLat = ds['starty'][:]


        # phi = 90 - latitude
        phi1 = np.deg2rad(90.0 - startLat)
        phi2 = np.deg2rad(90.0 - (startLat+grid.dx))

        # theta = longitude
        theta1 = np.deg2rad(startLon)
        theta2 = np.deg2rad(startLon+grid.dx)

        kmPerLat = sqrt((phi1 - phi2)**2 + (theta1- theta2)**2)*6371
        # distance = earth radius * radians
        
        # distance = earth radius * radians
        Earth_radius = 6371
        km2rad= kmPerLat/Earth_radius
        gridDist = np.rad2deg(km2rad)
        
        #grid distance in km roughly
        grid.dx = (grid.dx*kmPerLat)/sqrt(gridDist**2)
        
    elif gridUnits.lower() =='km':
        grid.dx = ds['dx'] 
    elif gridUnits.lower() =='m':
        grid.dx = ds['dx']/1000 #convert m to km
    else:
        print('Unknown grid dx units: ' + gridUnits+'\n')
    
    #read valid grid point mask
    grid.mask = ds['mask'][:] 
    #read DEM
    grid.dem = ds['elev'][:]


    #set grid size variables
    (grid.nc,grid.nr) = np.shape(grid.lat)
    
    return grid
    

In [179]:
#read grid file
grid = readRawGrid(controlVars.gridName)
# set output grid
outGrid = grid

# calcTopoAspects:
* **Computes the cardinal slope facets and flat areas from a smoothed DEM that is computed internally here**

In [182]:
def calcTopoAspects(grid,parameters):
    """
     calcTopoAspects computes the cardinal slope facets and flat areas from 
      a smoothed DEM that is computed internally here. Generally follows Daly
      et al. (1994)

     Arguments:

     Input: grid, structure, the raw grid structure parameters, structure,
             structrure holding preprocessing parameters

     Output: aspects, structure, structure containing smoothed DEM, topographic
              facets, and smoothed topographic gradient arrays

     Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
     Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.
    """
    print('Computing topographic facets\n')

    #define local variable for facets

    intFacet = np.zeros(np.shape(grid.dem))-999.0

    #local variable for minimum gradient
    minGradient = parameters.minGradient

    #local variable for number of facets
    nFacets = 5

    #local variable for minimum size of facets
    nSmallFacet = parameters.smallFacet/(grid.dx**2);  #set size of smallest sloped facet to grid cells
    nSmallFlat = parameters.smallFlat/(grid.dx**2);    #set size of smallest flat facet to grid cells

    #check filter type and set filter
    if parameters.demFilterName.lower() =='daly':
        demFilter = np.array([[0, 0.125, 0], [0.125, 0.5, 0.125], [0, 0.125, 0]])

    else:
        print('Unknown Filter type'+ parameters.demFilterName+ '\n')

    #set local dem variable
    smoothElev = grid.dem.filled(0)


    #filter DEM using demFilterPasses
    for n in range(1,int(parameters.demFilterPasses)+1):
        smoothElev= ndimage.convolve(smoothElev, demFilter, mode='wrap', cval=0)


    #workaround for Octave compatibility
    #this gives slightly different aspect and facet results from the
    #gradientm function in Matlab, but overall is very similar
    (gradEast,gradNorth) = np.gradient(np.transpose(smoothElev),int((grid.dx*1000).filled())) #convert km to m
    aspect = 270-(360/(2*math.pi))*np.arctan2(gradNorth,gradEast)
    aspect[aspect>360] = aspect[aspect>360]-360   

    #transpose aspect, gradients
    aspect = np.transpose(aspect)
    gradNorth = np.transpose(gradNorth)
    gradEast = np.transpose(gradEast)

    #define flat facets 
    flat = (abs(gradNorth)<minGradient) & (abs(gradEast)<minGradient)

    #define cardinal direction facets
    north = (aspect>315) | (aspect<=45)  
    east = (aspect>45) & (aspect<=135) 
    south = (aspect>135) & (aspect<=225) 
    west = (aspect>225) & (aspect<=315) 

    intFacet[north] = 1
    intFacet[east] = 2
    intFacet[south] = 3
    intFacet[west] = 4
    intFacet[flat] = 5

    #character array of facets
    charFacets=[]

    #character array of facets
    for i in range(0,int(np.max(intFacet))):

        if i==0:
            charFacets.append('North') 
        elif i==1:
            charFacets.append('East') 
        elif i== 2:
            charFacets.append('South') 
        elif i== 3:
            charFacets.append('West') 
        elif i== 4:
            charFacets.append('Flat') 
        else:
            print('Unknown Facet')

    #local variable for number of facets
    #for North, West, east, South facets

    #merge small facets
    # find all objects for each aspect and merge small ones into nearby larger
    # Facets using 4- (flat) or 8-connectivity (slopes)

    nFacets=4
    for i in range(0,nFacets):

        print('Merging Facet '+charFacets[i] + '\n')
    #     if i+1 < nFacets:
    #         connectivity = 8
    #     else:
    #         connectivity = 4

        binary = intFacet
        binary= np.where(binary==(i+1),1,0)

        # s = ndimage.generate_binary_structure(2,2)
        # imageObjects = ndimage.label(binary, s) # same image_binary as above
        #imageObjects = measure.label(binary, connectivity) # same image_binary as above

        imageObjects = measure.label(binary) # same image_binary as above
        props = measure.regionprops_table(imageObjects, properties=('area','bbox','minor_axis_length','major_axis_length'))
        stats=pd.DataFrame(props)

        if i < (nFacets+1):
            minSize = nSmallFacet
        else: #flats need to be larger as small flats may behave like nearby slopes
            minSize = nSmallFlat

        #number of objects for current facet
        nobj = len(stats)

        for n in range(0,nobj):

           #if the current object is too small        
            if stats.iloc[n]['area'] < minSize:
            #find the west and south side bounding points
                westPoints =  [round(stats.iloc[n]['bbox-0']), round(stats.iloc[n]['bbox-2'])]
                southPoints = [round(stats.iloc[n]['bbox-1']), round(stats.iloc[n]['bbox-3'])]

                #if the bounding point is outside the grid size, set to grid max
                #dimensions for columns and rows
                if max(westPoints)>grid.nc:
                    westPoints[1] = grid.nc

                if max(southPoints)>grid.nr:
                    southPoints[1] = grid.nr


                #define north and east bounding points
                northPoints = southPoints
                eastPoints = westPoints


                #find the grid cells along the four bounding lines
                westPixels = intFacet[westPoints[0]:westPoints[1]+1,round(stats.iloc[n]['bbox-1'])]
                southPixels = intFacet[round(stats.iloc[n]['bbox-0']),southPoints[0]:southPoints[1]+1]
                #north and east bounding points are defined by the opposite of the
                #west and south bounding points
                eastPixels = intFacet[eastPoints[0]:eastPoints[1]+1,southPoints[1]-1]
                northPixels = intFacet[westPoints[1]-1,northPoints[0]:northPoints[1]+1]

                #find the mode of the facets on the bounding lines
                modeWest = mode(westPixels)
                modeEast = mode(eastPixels)
                modeSouth = mode(southPixels)
                modeNorth = mode(northPixels)

                #find the grid cells of the current facet object
                inds = np.where(imageObjects==n+1)


                #merge current object into appropriate facet based on bounding
                #line most common facet
                if (modeWest != 5 and modeWest != i+1):
                    intFacet[inds] = modeWest #merge into west-facing slope
                elif (modeSouth !=5 and modeSouth != i+1):
                    intFacet[inds] = modeSouth #merge into south-facing slope
                elif (modeEast !=5 and modeEast != i+1):
                    intFacet[inds] = modeEast #merge into east-facing slope
                elif (modeNorth !=5 and modeNorth != i+1):
                    intFacet[inds] = modeNorth #merge into north-facing slope
                else:  #if object cannot merge into slope, default merge into flat
                    intFacet[inds] = 5 #merge into flat area


            #end object size if-statement
        #end object loop
    #end facet loop
    # merge narrow flat areas that are a likely ridge
    #assume these behave like nearby slopes
    #merge into slopes on the south and west sides only
    print('Merging narrow flats\n')

    #create a binary image of only flat pixels
    f = 5
    binary = intFacet
    binary= np.where(binary==f,1,0)
    #identify connected pixels
    # s = ndimage.generate_binary_structure(2,1)
    # imageObjects, c = ndimage.label(binary, s) # same image_binary as above
    imageObjects = measure.label(binary) 


    #generate statistics about all object features
    props_flat = measure.regionprops_table(imageObjects, properties=('area','bbox','minor_axis_length','major_axis_length','orientation'))                                                                  
    flats=pd.DataFrame(props_flat)


    #loop through all flat objects
    for i in range(0,len(flats)):
        #if they are very narrow
        if flats.iloc[i]['major_axis_length']/flats.iloc[i]['minor_axis_length'] > parameters.narrowFlatRatio:
            #find the west and south side bounding points
            westPoints =  [math.floor(flats.iloc[i]['bbox-0']), math.floor(flats.iloc[i]['bbox-2'])]
            southPoints = [math.floor(flats.iloc[i]['bbox-1']), math.floor(flats.iloc[i]['bbox-3'])]

            westPoints = [1 if item == 0 else item for item in westPoints]         
            southPoints = [1 if item == 0 else item for item in southPoints]   



            flats[flats.loc[:,['bbox-0','bbox-1']]<1] = 1



            #find what facets the bounding line grid cells belong to                        
            westPixels = intFacet[westPoints[0]:westPoints[1]+1, math.floor(flats.iloc[n]['bbox-1'])]
            southPixels = intFacet[math.floor(flats.iloc[n]['bbox-0']), southPoints[0]:southPoints[1]+1]


            #find the mode of the facets on the west and south sides
            modeWest = mode(westPixels)
            modeSouth = mode(southPixels)

            #find current object grid points
            inds = np.where(imageObjects==i)

            #merge if the mode of the south pixels is not flat and if the
            #current flat is oriented less than 45 degrees
            if abs(flats.iloc[i]['orientation']<=45 and modeSouth != 5):
                #merge into South slope
                intFacet[inds] = modeSouth
            else:  #else merge into whatever is along the west bounding box
                #merge into West slope
                intFacet[inds] = modeWest

    #set smoothElev non-land pixels to missing
    smoothElev[grid.mask<=0] = -999
    gradNorth[grid.mask<=0] = -999
    gradEast[grid.mask<=0] = -999

    intFacet[grid.mask<=0] = -999

    class structtype():
        pass
    aspects = structtype()

    #define output varibles
    aspects.smoothDEM = smoothElev
    aspects.gradNorth = gradNorth
    aspects.gradEast = gradEast
    aspects.facets = intFacet
    aspects.facets[109]
    return aspects


In [183]:
#create topographic gradients and facets(integer numbers) (e.g. Daly et al. 1994)
outGrid.aspects = calcTopoAspects(grid,parameters)


Computing topographic facets

Merging Facet North

Merging Facet East

Merging Facet South

Merging Facet West

Merging narrow flats



C:\Users\askarzam\AppData\Local\Temp/ipykernel_14512/656281968.py:228: RuntimeWarning: invalid value encountered in double_scalars
  if flats.iloc[i]['major_axis_length']/flats.iloc[i]['minor_axis_length'] > parameters.narrowFlatRatio:
C:\Users\askarzam\AppData\Local\Temp/ipykernel_14512/656281968.py:228: RuntimeWarning: divide by zero encountered in double_scalars
  if flats.iloc[i]['major_axis_length']/flats.iloc[i]['minor_axis_length'] > parameters.narrowFlatRatio:


In [17]:
outGrid.aspects.gradNorth

array([[-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
        -5.63481419e-03, -5.84545639e-03, -4.93675500e-03],
       [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
        -1.11749170e-03, -2.67126414e-03, -3.10026242e-03],
       [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
         5.13381692e-03,  1.84463073e-03, -3.66258967e-04],
       ...,
       [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
        -1.32500502e-02, -9.49498178e-03, -5.33731721e-03],
       [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
        -1.50142450e-02, -1.15514283e-02, -7.19955037e-03],
       [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
        -1.60101522e-02, -1.28567663e-02, -8.49111176e-03]])

# calcDistToCoast:
* **Computes the distance to the nearest ocean masked grid cell using the input grid DEM and mask**

In [184]:
def calcDistToCoast(grid):
    
    """
    calcDistToCoast computes the distance to the nearest ocean masked grid 
    cell using the input grid DEM and mask.  Uses a search length of
    searchLength km.  Any grid cells that do not have ocean within that
    search length have the distanceToCoast set to searchLength.  Generally
    follows Daly et al. (2002)

    Arguments:

     Input: grid, structure, the raw grid structure searchLength,
     float, the maximum search distance to compute coastal distance

     Output: distToCoast, array, array of coastal distances for domain, computed at
                          valid land pixels only

         Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
         Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """

    #Note this is a time consuming routine as written.  There are likely
    #opportunities for speed-up for large domains.
       
    

    #print status
    print('Computing distance to coast\n')

    #find all land points in domain
    landPts = grid.mask > 0
    #lat of land points
    latLand = grid.lat[landPts].filled()

    #lon of land points
    lonLand = grid.lon[landPts].filled()
    #how many land points
    lenLand = len(lonLand)
    #array indices of land points


    landInds = np.argwhere(grid.mask > 0)

    #define output variable, set to missing
    distToCoast = np.zeros([grid.nc,grid.nr])-999.0

    # #find all (if any) ocean pixels
    oceanValid = grid.mask == -1
    latOcean = grid.lat[oceanValid]
    lonOcean = grid.lon[oceanValid]  
    
 # Here's a vectorized method leveraging the NumPy ufuncs (haversine func) to replace those math-module funcs 
    #so that we would operate on entire arrays in one go    

    if oceanValid.any():
            
        # convert all latitudes/longitudes from degrees to radians
        beg_coord= np.deg2rad(np.transpose([latLand, lonLand]))
        end_coords= np.deg2rad(np.transpose([latOcean, lonOcean]))   
        # calculate haversine
        lat = end_coords[:,0] - beg_coord[:,0,None]
        lng = end_coords[:,1] - beg_coord[:,1,None]

        # Compute the "cos(lat1) * cos(lat2) * sin(lng * 0.5) ** 2" 
        add0 = np.cos(beg_coord[:,0,None])*np.cos(end_coords[:,0])* np.sin(lng * 0.5) ** 2
        # Add into "sin(lat * 0.5) ** 2" 
        d = np.sin(lat * 0.5) ** 2 +  add0

        Earth_radius = 6371
        h = 2 * Earth_radius * np.arcsin(np.sqrt(d))
        #find nearest ocean pixels
        mindists= h.min(1)
        
        #loop through all land points
        for pt in range(0,lenLand):
            distToCoast[landInds[pt,0],landInds[pt,1]]  = mindists[pt]
        

        #find maximum distance computed
        maxDist = np.max(distToCoast[grid.mask == 1])

        #set all non-computed valid land points to maxDist 
        distToCoast [(grid.mask == 1) & (distToCoast == -999)] = maxDist
    else:
        distToCoast = distToCoast + 999
        
    return distToCoast

In [185]:
#calculate distance to coastline (km) (e.g. Daly et al. 2002,2008)
outGrid.distToCoast = calcDistToCoast(grid)

Computing distance to coast



# calcPositions:
* **Computes the topographic position of each grid cell and determines what layer of the atmosphere**

In [195]:
def calcPositions(grid,searchLength,inversionHeight):
    
    """
    calcPositions computes the topographic position of each grid cell and 
     determines what layer of the atmosphere (inversion or free) each grid cell
     lies in

     Arguments:

     Input: grid,structure, the raw grid structure searchLength float, search length (grid cells) for local DEM
                              calculations
      inversionHeight, float, height of inversions (m). grid cells with
                              topographic position above inversionHeight are 
                              in free atmosphere, those below are in areas 
                              prone to inversions

     Output: positions, structure, structure containing topographic position and
                            inversion layer arrays

             Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
             Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    print('Computing topographic positions\n')


    #run through all grid points, determine local inversion height, which grid
    #points might be in an inversion if present (e.g. layer 1 & 2 temperature
    #atmosphere in Daly et al. 2002)

    # pass 1 - find minimum elevation for all cells within search radius
    # (searchLength)
    #define minElev array
    minElev = np.zeros([grid.nc,grid.nr])
    #define inversion layer mask
    class structtype():
        pass
    positions = structtype()
    positions.layerMask = np.zeros([grid.nc,grid.nr])-999.0

    #define local dem variable
    dem = grid.dem.filled()
    mask= grid.mask.filled()
    # loop over all grid points
    for i in range(0,grid.nc):
        for j in range (0,grid.nr):
            #if we are at a valid land point
            if (mask[i,j] == 1):
                #find column range indices
                cRange = [max([0, i-searchLength]), min([i+searchLength, grid.nc])]
                cRange= [int(a) for a in cRange]

                #find row range indices
                rRange = [max([0, j-searchLength]),min([j+searchLength, grid.nr])]
                rRange= [int(a) for a in rRange]

                #find valid DEM points within search box
                demValid = mask[cRange[0]:cRange[1], rRange[0]:rRange[1]]==1
                #subset dem using search box
                demSub = dem[cRange[0]:cRange[1], rRange[0]:rRange[1]]
                #find minimum valid land elevation in box
                minElev[i,j] = min(demSub[demValid])


    # pass 2 - now find the mean of the min_elev grid, add inversion height, 
    #then determine layer-1 or 2 for full-res elevation

    #define meanElev, inversionBase and topoPosition arrays
    meanElev = np.zeros([grid.nc,grid.nr])
    inversionBase = meanElev
    positions.topoPosition = meanElev-999.0

    #loop over all grid points
    for i in range(0,grid.nc): 
        for j in  range(0,grid.nr): 
            #if we are at a valid land point
            if mask[i,j] == 1:
                #find row range indices
                rRange = [max([1, j-searchLength]), min([j+searchLength, grid.nr])]
                rRange= [int(a) for a in rRange]
                #find column range indices
                cRange = [max([1, i-searchLength]), min([i+searchLength, grid.nc])]
                cRange= [int(a) for a in cRange]
                #find valid DEM points within search box
                demValid = mask[cRange[0]:cRange[1], rRange[0]:rRange[1]]==1
                #subset minElev array using search box
                demSub = minElev[cRange[0]:cRange[1], rRange[0]:rRange[1]]
                #find mean of minElev subset over valid land dem cells
                meanElev[i, j] = mean(demSub[demValid])
                #compute inversion base
                inversionBase[i,j] = inversionHeight + meanElev[i,j]
                #compute topograpic position
                positions.topoPosition[i,j] = dem[i,j] - meanElev[i,j]


    #set atmospheric layers (1= inversion layer, 2=free atmosphere);
    positions.layerMask[tuple([dem<=inversionBase] and [dem>0])] = 1
    positions.layerMask[dem>inversionBase] = 2

    
    return positions



In [196]:
#calculate two-layer atmospheric position (layer 1 or 2) and the
# topographic position (m) (e.g. Daly et al. 2002,2008)
outGrid.positions = calcPositions(grid,parameters.layerSearchLength,parameters.inversionHeight)
outGrid.positions.layerMask[0]

Computing topographic positions



array([-999., -999., -999., -999., -999., -999., -999., -999., -999.,
       -999., -999., -999., -999., -999., -999., -999., -999., -999.,
       -999., -999., -999., -999., -999., -999., -999., -999., -999.,
       -999., -999., -999., -999., -999., -999., -999., -999., -999.,
       -999., -999., -999., -999., -999., -999., -999., -999., -999.,
       -999., -999., -999., -999., -999., -999., -999., -999., -999.,
       -999., -999., -999., -999., -999., -999., -999., -999., -999.,
       -999., -999., -999., -999., -999., -999.,    1.,    1.,    1.,
          1.,    2.,    2.,    2.,    2.,    2.,    2.,    2.,    2.,
          2.,    2.,    2.,    2.,    2.,    2.,    2.,    2.,    2.,
          1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,    1.,
          2.,    2.,    2.,    2.,    2.,    2.,    2.,    2.,    2.,
          2.,    2.,    2.,    2.,    2.,    2.,    2.,    2.,    2.,
          2.,    2.,    2.,    2.,    1.,    1.,    1.,    1.,    2.,
          2.,    2.]

In [189]:
def outputGrid(controlVars,outGrid): 
    """
    outputGrid saves the domain grid structure into a netcdf file

     Arguments:

      Input: controlVars, structure   , structure containing control variables
       outGrid   , structure, structure containing processed domain grid and
       geophysical attributes

      Output:None

     Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
     Email : anewman@ucar.edu & mozhgana@ucar.edu

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any lgeopater version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.
    """
          
    
    print('Outputting processed grid to netcdf file\n')
    
    # first copy raw grid to output grid
    print("copy %s %s" %(controlVars.gridName,controlVars.outputName))
    shutil.copyfile(controlVars.gridName, controlVars.outputName)
    
    
    try: ncfile.close()  # just to be safe, make sure dataset is not already open.
    except: pass
    
    
    ncfile = Dataset(controlVars.outputName,mode='w',format='NETCDF4_CLASSIC') 
#     ncfile = Dataset('../data/grid/exampleTIERGrid_test.nc',mode='w',format='NETCDF4_CLASSIC') 
    # Create the dimensions
    ncfile.createDimension('y',  outGrid.nc)     
    ncfile.createDimension('x',  outGrid.nr) 
    ####################################
    #output smoothed elevation

    # Create the attributes    
    smooth_elev = ncfile.createVariable('smooth_elev', np.float64,('y','x'), fill_value= -999.0)
    smooth_elev.setncattr('name','Smoothed DEM')
    smooth_elev.setncattr('long_name','Smoothed Digital Elevation Map')
    smooth_elev.setncattr('units','m')
    print(smooth_elev)
     # Appends data along unlimited dimension   
    smooth_elev[:,:] = outGrid.aspects.smoothDEM 

    #####################
    # output smoothed gradient terms for book keeping
    # north - south graident

    # Create the attributes    
    gradient_n_s = ncfile.createVariable('gradient_n_s', np.float64,('y','x'), fill_value= -999.0)
    gradient_n_s.setncattr('name','N-S Gradient')
    gradient_n_s.setncattr('long_name','Smoothed North-South Gradient (North facing positive)')
    gradient_n_s.setncattr('units','km/km')
    print(gradient_n_s)
     # Appends data along unlimited dimension
    gradient_n_s[:,:] = outGrid.aspects.gradNorth

    # west - east gradient

    # Create the attributes    
    gradient_w_e = ncfile.createVariable('gradient_w_e', np.float64,('y','x'), fill_value= -999.0)
    gradient_w_e.setncattr('name','W-E Gradient')
    gradient_w_e.setncattr('long_name','Smoothed West-East Gradient (West facing positive)')
    gradient_w_e.setncattr('units','km/km')
    print(gradient_w_e)
     # Appends data along unlimited dimension
    gradient_w_e[:,:] = outGrid.aspects.gradEast
    ############################################


    # output facets

    # Create the attributes    
    facet = ncfile.createVariable('facet', np.float64,('y','x'), fill_value= -999.0)
    facet.setncattr('name','integer facet')
    facet.setncattr('long_name','integer facet: 1=N,2=E,3=S,4=W,5=Flat generally following Daly et al. 1994')
    facet.setncattr('units','-')
    print(facet)
     # Appends data along unlimited dimension
    facet[:,:] = outGrid.aspects.facets
    ############################################

    # output distance to coast

    # Create the attributes    
    dist_to_coast = ncfile.createVariable('dist_to_coast', np.float64,('y','x'), fill_value= -999.0)
    dist_to_coast.setncattr('name','Distance to coast')
    dist_to_coast.setncattr('long_name','Distance to closest water point in DEM (e.g. Daly et al. 2002)')
    dist_to_coast.setncattr('units','km')
    print(dist_to_coast)
     # Appends data along unlimited dimension
    dist_to_coast[:,:] = outGrid.distToCoast
    ############################################

    # output topographic position

    # Create the attributes    
    topo_position = ncfile.createVariable('topo_position', np.float64,('y','x'), fill_value= -999.0)
    topo_position.setncattr('name','Topographic Position')
    topo_position.setncattr('long_name','Grid point topographic position relative to nearby smoothed DEM cells following Daly et al. 2002')
    topo_position.setncattr('units','m')
    print(topo_position)
     # Appends data along unlimited dimension
    topo_position[:,:] = outGrid.positions.topoPosition
    ############################################

    # output inversion layer

    # Create the attributes    
    inversion_layer = ncfile.createVariable('inversion_layer', np.float64,('y','x'), fill_value= -999.0)
    inversion_layer.setncattr('name','Atmospheric Layer')
    inversion_layer.setncattr('long_name','Simple two-layer atmospheric position (1=inversion, 2=free) from Daly et al. 2002')
    inversion_layer.setncattr('units','-')
    print(inversion_layer)
     # Appends data along unlimited dimension
    inversion_layer[:,:] = outGrid.positions.layerMask


In [190]:
#output to processed grid file
outputGrid(controlVars,outGrid)

Outputting processed grid to netcdf file

copy C:\Users\askarzam\Downloads\TIER-master\data\grid\exampleGridRaw.nc C:\Users\askarzam\Downloads\TIER-master\data\grid\exampleTIERGrid_1.nc
<class 'netCDF4._netCDF4.Variable'>
float64 smooth_elev(y, x)
    _FillValue: -999.0
    name: Smoothed DEM
    long_name: Smoothed Digital Elevation Map
    units: m
unlimited dimensions: 
current shape = (136, 128)
filling on
<class 'netCDF4._netCDF4.Variable'>
float64 gradient_n_s(y, x)
    _FillValue: -999.0
    name: N-S Gradient
    long_name: Smoothed North-South Gradient (North facing positive)
    units: km/km
unlimited dimensions: 
current shape = (136, 128)
filling on
<class 'netCDF4._netCDF4.Variable'>
float64 gradient_w_e(y, x)
    _FillValue: -999.0
    name: W-E Gradient
    long_name: Smoothed West-East Gradient (West facing positive)
    units: km/km
unlimited dimensions: 
current shape = (136, 128)
filling on
<class 'netCDF4._netCDF4.Variable'>
float64 facet(y, x)
    _FillValue: -999.

# createPrecipitationStationList:
* **Creates the precipitation station list**

In [191]:
def createPrecipitationStationList(controlVars,grid):
    """

 createPrecipitationStationList creates the precipitation station list 
 used in TIER processing
 TIER - Topographically InformEd Regression

 Arguments:

 Input:controlVars, structure, structure containing preprocessing control variables
  grid,        structure, structure containing DEM variables

 Output:none, function writes to a file

     Author: Andrew Newman, NCAR/RAL and Mozhgan A. Farahani SIParCs Intern
     Email : anewman@ucar.edu & mozhgana@ucar.edu
 
 Copyright (C) 2019 University Corporation for Atmospheric Research

 This file is part of TIER.

 TIER is free software: you can redistribute it and/or modify
 it under the terms of the GNU General Public License as published by
 the Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 TIER is distributed in the hope that it will be useful,
 but WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 GNU General Public License for more details.

 You should have received a copy of the GNU General Public License
 along with TIER.  If not, see <https://www.gnu.org/licenses/>.
"""


    #print status
    print('Compiling precipitation station list\n')

    #define local variables
    nr = grid.nr
    nc = grid.nc

    #find nearest grid point
    #transform grid to 1-d arrays for computational convenience

    lon1d= np.reshape(outGrid.lon, nr*nc)
    lat1d = np.reshape(outGrid.lat, nr*nc)

    facet1d = np.reshape(outGrid.aspects.facets,nr*nc)
    distToCoast1d = np.reshape(outGrid.distToCoast,nr*nc )
    layerMask1d = np.reshape(outGrid.positions.layerMask,nr*nc)
    topoPosition1d = np.reshape(outGrid.positions.topoPosition,nr*nc)

    #create list of stations in directory
    # folder path
    dir_path = controlVars.stationPrecipPath

    # list to store files
    fileList = []
    # Iterate directory
    for file in os.listdir(dir_path):
        # check only text files
        if file.endswith('.nc'):
            fileList.append(file)

    #number of stations
    nSta = len(fileList)


    class structtype():
        pass
    station = structtype()
    lat = []
    lon=[]
    elev=[]
    # read lat,lon,elevation from station timeseries file
    for f in range(0,nSta):
        stationName= '%s\%s'%(controlVars.stationPrecipPath,fileList[f])
        #loading dataset by passing a NetCDF file path
        sn = Dataset(stationName)
        #read latitude and longitude
        lat.append(sn['latitude'][:])
        station.lat= lat
        lon.append(sn['longitude'][:])
        station.lon = lon
        elev.append(sn['elevation'][:])
        station.elev = elev


    #open output station list file
    #print header to output file
    sidOut = open(controlVars.stationPrecipListName,'w')
    sidOut.write("NSITES %d\n"%nSta)

    sidOut = open(controlVars.stationPrecipListName,'a')
    sidOut.write("%s %s %s %s %s %s %s %s %s" %('#STNID\t','LAT\t','LON\t','ELEV\t',\
                                                'ASP\t','DIST_COAST\t' ,'INVERSION\t','TOPO_POS\t' ,'STN_NAME\n'))


    #loop through all stations and find the nearest grid point for geophysical
    #attributes
    for i in range(0,nSta):
        stationId = fileList[i].split(".")[0]
        #compute distances and indices of nearest grid points
        ix = np.argsort(np.sqrt((lat1d[:,None]-station.lat[i])**2 + (lon1d[:,None]-station.lon[i])**2), axis=None)

        #if nearest grid point is valid
        if np.isnan(facet1d[ix[0]]) == False :
            #output geophysical attributes to station file

            sidOut = open(controlVars.stationPrecipListName,'a')
            sidOut.write('%s, %9.5f, %11.5f, %7.2f, %d, %8.3f, %d, %8.3f, %s\n'%(stationId,station.lat[i],station.lon[i],\
                             station.elev[i],facet1d[ix[0]],distToCoast1d[ix[0]],layerMask1d[ix[0]],topoPosition1d[ix[0]],\
                                                                                 stationId))
        else: #if not valid
            #find the nearest valid point for all attributes

            nearestValid = np.where(np.isnan(facet1d[ix]) == False)

            #output geophysical attributes to station file
            sidOut.write('%s, %9.5f, %11.5f, %7.2f, %d, %8.3f, %d, %8.3f, %s\n'%(stationId,station.lat[i],station.lon[i],\
                            station.elev[i],facet1d[ix(nearestValid[0])],distToCoast1d[ix[nearestValid[0]]],\
                            layerMask1d[ix[nearestValid[0]]],topoPosition1d[ix[nearestValid[0]]],stationId))     


    #close output list file
    sidOut.close()


In [192]:
#create station list files for input to main TIER program
#precipitation
createPrecipitationStationList(controlVars,outGrid)

Compiling precipitation station list



# createTemperatureStationList:
* **Creates the temperature station list**

In [193]:
def createTemperatureStationList(controlVars,grid):
    """
     createTemperatureStationList creates the temperature station list 
     used in TIER processing
     TIER - Topographically InformEd Regression

     Arguments:

     Input:controlVars, structure, structure containing preprocessing control variables
      grid,        structure, structure containing DEM variables

     Output:none, function writes to a file

     Author: Andrew Newman, NCAR/RAL
     Email : anewman@ucar.edu
     Postal address:
         P.O. Box 3000
         Boulder,CO 80307

     Copyright (C) 2019 University Corporation for Atmospheric Research

     This file is part of TIER.

     TIER is free software: you can redistribute it and/or modify
     it under the terms of the GNU General Public License as published by
     the Free Software Foundation, either version 3 of the License, or
     (at your option) any later version.

     TIER is distributed in the hope that it will be useful,
     but WITHOUT ANY WARRANTY; without even the implied warranty of
     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
     GNU General Public License for more details.

     You should have received a copy of the GNU General Public License
     along with TIER.  If not, see <https://www.gnu.org/licenses/>.

    """
    #print status
    print('Compiling temperature station list\n')

    #define local variables
    nr = grid.nr
    nc = grid.nc

    #find nearest grid point
    #transform grid to 1-d arrays for computational convenience

    lon1d= np.reshape(outGrid.lon, nr*nc)
    lat1d = np.reshape(outGrid.lat, nr*nc)

    facet1d = np.reshape(outGrid.aspects.facets,nr*nc)
    distToCoast1d = np.reshape(outGrid.distToCoast,nr*nc )
    layerMask1d = np.reshape(outGrid.positions.layerMask,nr*nc)
    topoPosition1d = np.reshape(outGrid.positions.topoPosition,nr*nc)

    #create list of stations in directory
    # folder path
    dir_path = controlVars.stationTempPath

    # list to store files
    fileList = []
    # Iterate directory
    for file in os.listdir(dir_path):
        # check only text files
        if file.endswith('.nc'):
            fileList.append(file)

    #number of stations
    nSta = len(fileList)


    class structtype():
        pass
    station = structtype()
    lat = []
    lon=[]
    elev=[]
    # read lat,lon,elevation from station timeseries file
    for f in range(0,nSta):
        stationName= '%s\%s'%(controlVars.stationTempPath,fileList[f])
        #loading dataset by passing a NetCDF file path
        sn = Dataset(stationName)
        #read latitude and longitude
        lat.append(sn['latitude'][:])
        station.lat= lat
        lon.append(sn['longitude'][:])
        station.lon = lon
        elev.append(sn['elevation'][:])
        station.elev = elev


    #open output station list file
    #print header to output file
    sidOut = open(controlVars.stationTempListName,'w')
    sidOut.write("NSITES %d\n"%nSta)

    sidOut = open(controlVars.stationTempListName,'a')
    sidOut.write("%s %s %s %s %s %s %s %s %s" %('#STNID\t','LAT\t','LON\t','ELEV\t','ASP\t','DIST_COAST\t' ,'INVERSION\t','TOPO_POS\t' ,'STN_NAME\n'))


    #loop through all stations and find the nearest grid point for geophysical
    #attributes
    for i in range(0,nSta):
        stationId = fileList[i].split(".")[0]
        #compute distances and indices of nearest grid points
        ix = np.argsort(np.sqrt((lat1d[:,None]-station.lat[i])**2 + (lon1d[:,None]-station.lon[i])**2), axis=None)

        #if nearest grid point is valid
        if np.isnan(facet1d[ix[0]]) == False :
            #output geophysical attributes to station file

            sidOut = open(controlVars.stationTempListName,'a')
            sidOut.write('%s, %9.5f, %11.5f, %7.2f, %d, %8.3f, %d, %8.3f, %s\n'%(stationId,station.lat[i],station.lon[i],\
                             station.elev[i],facet1d[ix[0]],distToCoast1d[ix[0]],layerMask1d[ix[0]],topoPosition1d[ix[0]],stationId))
        else: #if not valid
            #find the nearest valid point for all attributes

            nearestValid = np.where(np.isnan(facet1d[ix]) == False)

            #output geophysical attributes to station file
            sidOut.write('%s, %9.5f, %11.5f, %7.2f, %d, %8.3f, %d, %8.3f, %s\n'%(stationId,station.lat[i],station.lon[i],\
                            station.elev[i],facet1d[ix(nearestValid[0])],distToCoast1d[ix[nearestValid[0]]],\
                            layerMask1d[ix[nearestValid[0]]],topoPosition1d[ix[nearestValid[0]]],stationId))     


    #close output list file
    sidOut.close()

In [194]:
#temperature
createTemperatureStationList(controlVars,outGrid);


Compiling temperature station list



# Required preprocessing steps complete

* Optional:
Users could add preprocessing for spatially distributed lapse rates. The
TIER release contains an example temperature lapse rate grid, but leaves
the generation as an excercise for the user.